In [ ]:
# helpers
from pprint import pprint


# Basic NLP operations

1. Lemmatization
1. Part-of-speech (POS) tagging
1. Dependency parsing
1. Rule-based pattern matching
1. Named entity recognition (NER)
1. Word embeddings

## Tools
- Python 3
- spaCy

In [ ]:
#
# Lemmatization
#

import spacy

text = u'''She goes for a walk every day.
           He was going to the cinema in the evening.
           They have already gone to work.
           I went home.'''
word_to_find = u'go'

nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

result = [(token.lemma_, token.text)
          for token in doc
          if token.lemma_ == word_to_find]

print(f'The word {word_to_find} appears {len(result)} times:')
pprint(result, width=25)

In [ ]:
#
# Part-of-speech (POS) tagging
#

import spacy

text = u'The sky above the port was the color of television, tuned to a dead channel.'

nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

for token in doc:
    print(f'{token.text:<12} {token.pos_}')

In [ ]:
#
# Rule-based pattern matching
#


In [ ]:
#
# Dependency parsing
#


In [ ]:
#
# Named entity recognition (NER)
#


In [ ]:
#
# Word embeddings
#